# R IRIS Model

 * Wrap an R decision-tree model (using the ```rpart``` library) for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * R

```bash
pip install seldon-core
```

## Train locally
 

In [1]:
!Rscript train.R

Wrap model using s2i

In [2]:
!s2i build . seldonio/seldon-core-s2i-r:0.1 r-iris:0.1

---> Installing application source...
---> Installing dependencies ...
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/rpart_4.1-13.tar.gz'
Content type 'application/x-gzip' length 818290 bytes (799 KB)
downloaded 799 KB

* installing *source* package ‘rpart’ ...
** package ‘rpart’ successfully unpacked and MD5 sums checked
** libs
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-udh3fS/r-base-3.5.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c anova.c -o anova.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-udh3fS/r-base-3.5.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c anovapred.c -o anovapred.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/

installing to /usr/local/lib/R/site-library/rpart/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded
* DONE (rpart)

The downloaded source packages are in
	‘/tmp/RtmpEJku2A/downloaded_packages’
Build completed successfully


In [3]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 r-iris:0.1

17052b4b55973ee3849a4085e78ead575d1c47cbad0e11e28ea1792d4644f4b1


Send some random features that conform to the contract

In [4]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[4.05  4.709 8.119 1.895]]
RECEIVED RESPONSE:
data {
  names: "setosa"
  names: "versicolor"
  names: "virginica"
  ndarray {
    values {
      list_value {
        values {
          number_value: 0.0
        }
        values {
          number_value: 0.0217
        }
        values {
          number_value: 0.9783
        }
      }
    }
  }
}




In [5]:
!docker rm iris_predictor --force

iris_predictor


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [6]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


In [7]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [8]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [9]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [10]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

NAME:   seldon-core-crd
LAST DEPLOYED: Wed Mar 13 19:06:54 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     4s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s


NOTES:
NOTES: TODO


NAME:   seldon-core
LAST DEPLOYED: Wed Mar 13 19:06:59 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        0s

==> v1/Role
NAME

In [11]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-r:0.1 r-iris:0.1

---> Installing application source...
---> Installing dependencies ...
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/rpart_4.1-13.tar.gz'
Content type 'application/x-gzip' length 818290 bytes (799 KB)
downloaded 799 KB

* installing *source* package ‘rpart’ ...
** package ‘rpart’ successfully unpacked and MD5 sums checked
** libs
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-udh3fS/r-base-3.5.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c anova.c -o anova.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-udh3fS/r-base-3.5.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c anovapred.c -o anovapred.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/

** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded
* DONE (rpart)

The downloaded source packages are in
	‘/tmp/RtmpUwUqyr/downloaded_packages’
Build completed successfully


In [12]:
!kubectl create -f r_iris_deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [13]:
!kubectl rollout status deploy/r-iris-deployment-r-iris-predictor-0cbaec2

Waiting for deployment "r-iris-deployment-r-iris-predictor-0cbaec2" rollout to finish: 0 of 1 updated replicas are available...
deployment "r-iris-deployment-r-iris-predictor-0cbaec2" successfully rolled out


In [14]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

----------------------------------------
SENDING NEW REQUEST:

[[5.27  3.07  3.932 0.911]]
RECEIVED RESPONSE:
meta {
  puid: "p95chfqpbcb5m30vt1kdvlq5s8"
  requestPath {
    key: "r-iris-classifier"
    value: "r-iris:0.1"
  }
}
data {
  names: "setosa"
  names: "versicolor"
  names: "virginica"
  tensor {
    shape: 1
    shape: 3
    values: 0.0
    values: 0.9074
    values: 0.0926
  }
}




In [15]:
!minikube delete

🔥  Deleting "minikube" from virtualbox ...
💔  The "minikube" cluster has been deleted.
